In [1]:
import pandas as pd
import numpy as np
import data_completion_methods as daco
import util
PATH_SAVE = 'autocomplete_data/'
PATH_DEFAULT = 'default_values/'

# Completamiento de Datos
------------------------
Para enriquecer la base de datos del modelo de machine learning, crearemos varias bases de datos rellenando valores faltantes, segun varios criterios.

Estos datos se guardar'an en la carpeta `autocomplete_data`, en formato `.csv` separados por `,`, y codificados en `utf-8`. Cada fichero tendr'a como nombre:
    `autocomplete_data__C<XX>__<YY>.csv`
donde `<XX>` indica el n'umero del vector de caracter'isitcas que tienen los datos, y `<YY>` ser'ia el n'umero de la estrategia de rellenado de datos faltantes para la caracter'isticas `<XX>`.

In [2]:
# Cargamos los datos originales y mostramos el analisis estadistico
original_table = pd.read_csv('mortalidad_aprendizaje.csv', index_col=0)
original_table.describe()

,edad,peso,talla,imc,sexo,color_piel,asa,betabloqueadores,clopidogrel,heparina,...,trigliceridos,glicemia,ada,fraccion_eyeccion,leuco,hb,ck,ckmb,primera_asistencia_medica,estado_vital
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1340.000000,1185.000000,1353.000000,1353.000000,...,1212.000000,1319.000000,1353.000000,1264.000000,968.000000,1088.000000,789.000000,802.000000,1308.000000,1353.000000
mean,66.220990,73.026608,168.055432,25.793386,0.678492,0.900961,5.044030,24.940928,0.961567,0.992609,...,1.311304,7.595679,0.487066,46.238924,9.656612,123.789522,1212.449937,144.471322,198.576453,0.101996
std,12.655743,11.982885,5.863767,3.686736,0.467228,0.298825,5.174072,20.201098,0.192311,0.085684,...,0.720844,3.501403,0.500017,8.169612,5.518481,26.916457,730.361846,113.735988,219.119755,0.302754
min,13.000000,40.000000,120.000000,16.135000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,3.300000,0.000000,20.000000,5.200000,8.000000,1.000000,0.000000,1.000000,0.000000
25%,57.000000,65.000000,165.000000,23.121000,0.000000,1.000000,2.000000,12.000000,1.000000,1.000000,...,0.900000,5.500000,0.000000,42.000000,8.200000,125.000000,805.000000,85.000000,60.000000,0.000000
50%,67.000000,72.000000,168.000000,25.535000,1.000000,1.000000,4.000000,24.000000,1.000000,1.000000,...,1.200000,6.400000,0.000000,46.000000,9.400000,132.000000,1125.000000,120.500000,120.000000,0.000000
75%,75.000000,80.000000,172.000000,28.082000,1.000000,1.000000,5.000000,24.000000,1.000000,1.000000,...,1.500000,8.400000,1.000000,52.000000,10.300000,134.000000,1529.000000,165.000000,180.000000,0.000000
max,98.000000,150.000000,190.000000,58.273000,1.000000,1.000000,72.000000,99.000000,1.000000,1.000000,...,9.000000,43.000000,1.000000,79.000000,110.200000,238.000000,3999.000000,960.000000,1080.000000,1.000000


## Vector de Caracterisitcas 01
Solo consideraremos en este caso las carcteristicas siguientes:
- 'estado_vital'
- 'edad'
- 'sexo'
- 'peso'
- 'hipertension_arterial'
- 'diabetes_mellitus'
- 'tabaquismo'
- 'frecuencia_cardiaca'
- 'presion_arterial_sistolica',
- 'presion_arterial_diastolica',
- 'hb'
- 'creatinina'
- 'ckmb'
- 'fibrilacion_auricular'
- 'insuficiencia_renal_cronica'
- 'ieca'
- 'furosemida'
- 'otros_diureticos'
- 'clopidogrel'

In [3]:
caracteristicas_01 = [
'estado_vital',
'edad',
'sexo',
'peso',
'hipertension_arterial',
'diabetes_mellitus',
'tabaquismo',
'frecuencia_cardiaca',
# No encuentro : Respiration rate
'presion_arterial_sistolica',
'presion_arterial_diastolica',
'hb', # Al parecer es => glycated hemoglobin
'creatinina',
'ckmb',
# No encuentro : Troponin T
'fibrilacion_auricular',
'insuficiencia_renal_cronica', # Al parecer es => Acute renal failure
# No encuentro : RBBB
# No encuentro : LBBB
'ieca', # Al parecer es => ACE inhibitors
'furosemida', # Al parecer entraria en => Diuretics
'otros_diureticos', # Al parecer entraria en => Diuretics
# No encuentro : Aspirin
'clopidogrel',
]

### Completado 01
Llenaremos los valores faltantes de:
- 'ckmb' : en dependecia del 'sexo', y el 'infarto_agudo_miocardio', con los valores promedios del paper extranjero
- 'creatinina' : con el mismo valor por defecto cubano para todos
- 'hb' : en dependencia del sexo, pero con el valor por defecto cubano fijo

In [4]:
new_table = original_table.copy()[caracteristicas_01]
daco.complete_fixed__ckmb__01(original_table, new_table)
daco.complete_fixed__creatinina__01(original_table, new_table)
daco.complete_fixed__hb__01(original_table, new_table)
new_table.describe()

,estado_vital,edad,sexo,peso,hipertension_arterial,diabetes_mellitus,tabaquismo,frecuencia_cardiaca,presion_arterial_sistolica,presion_arterial_diastolica,hb,creatinina,ckmb,fibrilacion_auricular,insuficiencia_renal_cronica,ieca,furosemida,otros_diureticos,clopidogrel
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000
mean,0.101996,66.220990,0.678492,73.026608,0.849224,0.294900,0.478197,84.853659,125.937177,74.700665,127.637103,103.455285,154.460584,0.015521,0.032520,0.872136,0.182557,0.515152,0.961567
std,0.302754,12.655743,0.467228,11.982885,0.357963,0.456167,0.499709,14.946676,22.987900,14.060935,25.702221,59.439608,92.668129,0.123659,0.177443,0.334062,0.386446,0.499955,0.192311
min,0.000000,13.000000,0.000000,40.000000,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000,8.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,57.000000,0.000000,65.000000,1.000000,0.000000,0.000000,78.000000,120.000000,70.000000,127.000000,78.000000,95.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,0.000000,67.000000,1.000000,72.000000,1.000000,0.000000,0.000000,86.000000,130.000000,70.000000,133.000000,89.000000,171.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
75%,0.000000,75.000000,1.000000,80.000000,1.000000,1.000000,1.000000,88.000000,130.000000,80.000000,135.000000,113.000000,184.700000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,98.000000,1.000000,150.000000,1.000000,1.000000,1.000000,180.000000,240.000000,130.000000,238.000000,1036.000000,960.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
# Guardando datos rellenados 1
'''
Se rellenan los datos faltantes con
- Media total cubana [insertar art'iculo] de creatinina
- Media por sexos, cubana [insertar art'iculo] de hemoblogina
- Media por a~nos y cuadro de infarto, seg'un el paper extranjero [insertar art'iculo], de cmkb
'''
new_table.to_csv(f'{PATH_SAVE}completed_data__C01__01.csv')

### Completando 02

Llenaremos los valores faltantes de:
- 'ckmb' : en dependecia del 'sexo', y el 'infarto_agudo_miocardio', con los valores promedios del paper extranjero
- 'creatinina' : con el mismo valor por defecto cubano para todos
- 'hb' : en dependencia del sexo, con una distribuci'on normal con media igual al valor por defecto cubano, y desviaci'on 7

In [5]:
new_table = original_table.copy()[caracteristicas_01]
daco.complete_fixed__ckmb__01(original_table, new_table)
daco.complete_fixed__creatinina__01(original_table, new_table)
daco.complete_unfixed__hb__01(original_table, new_table)
new_table.describe()

,estado_vital,edad,sexo,peso,hipertension_arterial,diabetes_mellitus,tabaquismo,frecuencia_cardiaca,presion_arterial_sistolica,presion_arterial_diastolica,hb,creatinina,ckmb,fibrilacion_auricular,insuficiencia_renal_cronica,ieca,furosemida,otros_diureticos,clopidogrel
count,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000,1353.000000
mean,0.101996,66.220990,0.678492,73.026608,0.849224,0.294900,0.478197,84.853659,125.937177,74.700665,127.699385,103.455285,154.460584,0.015521,0.032520,0.872136,0.182557,0.515152,0.961567
std,0.302754,12.655743,0.467228,11.982885,0.357963,0.456167,0.499709,14.946676,22.987900,14.060935,25.951409,59.439608,92.668129,0.123659,0.177443,0.334062,0.386446,0.499955,0.192311
min,0.000000,13.000000,0.000000,40.000000,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000,8.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,57.000000,0.000000,65.000000,1.000000,0.000000,0.000000,78.000000,120.000000,70.000000,126.000000,78.000000,95.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,0.000000,67.000000,1.000000,72.000000,1.000000,0.000000,0.000000,86.000000,130.000000,70.000000,133.000000,89.000000,171.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
75%,0.000000,75.000000,1.000000,80.000000,1.000000,1.000000,1.000000,88.000000,130.000000,80.000000,135.000000,113.000000,184.700000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000
max,1.000000,98.000000,1.000000,150.000000,1.000000,1.000000,1.000000,180.000000,240.000000,130.000000,238.000000,1036.000000,960.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# Guardando datos rellenados 2
'''
Se rellenan los datos faltantes con
- Media total cubana [insertar art'iculo] de creatinina
- Media por sexos, cubana [insertar art'iculo] de hemoblogina con distribuci'on normal
- Media por a~nos y cuadro de infarto, seg'un el paper extranjero [insertar art'iculo], de cmkb
'''
# IMPORTANTE: Mantener comentada para no sobreescribir los datos ya guardados
#new_table.to_csv(f'{PATH_SAVE}completed_data__C01__02.csv')

### Completado 03

Llenaremos los valores faltantes de:
- 'ckmb' : en del 'infarto_agudo_miocardio', con los valores promedios de los datos reales sin anormalidades
- 'creatinina' : con el mismo valor por defecto cubano para todos
- 'hb' : en dependencia del sexo, con una distribuci'on normal con media igual al valor por defecto cubano, y desviaci'on 7

In [7]:
# Eliminemos los datos que tengan valores at'ipicos de `ckmb`.
clean_data = util.non_outlier_data(original_table, 'ckmb')
clean_data = original_table.where(util.z_score_abs(original_table['ckmb'])<=3).dropna(axis=0, how='all').reset_index(drop=True)
# Veamos las medias globales del `ckmb` tanto sin y con eliminaci'on de valores at'ipicos 
print('Medias globales sin eliminacion de valores atipicos: ', original_table['ckmb'].mean())
print('Medias globales con eliminacion de valores atipicos: ', clean_data['ckmb'].mean())
# Veamos alas diferencias existentes cuando se agrupa por `infarto_miocardio_agudo`
print('Medias con eliminacion de valores atipicos, considerando infarto = 0: ' , clean_data[clean_data['infarto_miocardio_agudo']==0]['ckmb'].mean())
print('Medias con eliminacion de valores atipicos, considerando infarto = 1: ' , clean_data[clean_data['infarto_miocardio_agudo']==1]['ckmb'].mean())

Medias globales sin eliminacion de valores atipicos:  144.4713216957606
Medias globales con eliminacion de valores atipicos:  133.81725888324874
Medias con eliminacion de valores atipicos, considerando infarto = 0:  136.15893108298172
Medias con eliminacion de valores atipicos, considerando infarto = 1:  112.1948051948052


In [ ]:
new_table = original_table.copy()[caracteristicas_01]
daco.complete_fixed__ckmb__02(original_table, new_table)
daco.complete_fixed__creatinina__01(original_table, new_table)
daco.complete_unfixed__hb__01(original_table, new_table)
new_table.describe()

In [ ]:
# Guardando datos rellenados 3
'''
Se rellenan los datos faltantes con
- Media total cubana [insertar art'iculo] de creatinina
- Media por sexos, cubana [insertar art'iculo] de hemoblogina con distribuci'on normal
- Media por cuadro de infarto, seg'un medias de datos sin valores at'ipicos, de cmkb
'''
# IMPORTANTE: Mantener comentada para no sobreescribir los datos ya guardados
new_table.to_csv(f'{PATH_SAVE}completed_data__C01__03.csv')

## Caracteristicas 02

De todas las caracter'isticas que tengan m'as de 1300 datos completos, se hace una eliminaci'on por filas de aquellas que tengan datos incompletos. De esta reducci'on nos quedamos con 62 caracter'isticas y 1256 registros

In [4]:
counts = original_table.describe().loc['count']
columns = counts[counts >= 1300].index
print(f'Cantidad de caracteristicas: {len(columns)}')
print(columns)

Cantidad de caracteristicas: 62
Index(['edad', 'peso', 'talla', 'imc', 'sexo', 'color_piel', 'asa',
       'clopidogrel', 'heparina', 'estatinas', 'estreptoquinasa_recombinante',
       'reperfusion', 'coronariografia', 'escala_grace', 'furosemida',
       'nitratos', 'anticoagulantes', 'anticalcicos', 'ieca',
       'otros_diureticos', 'lugar_trombolisis', 'atencion_inicial',
       'horario_llegada', 'ecg_previo', 'scacest', 'depresion_st',
       'depresion_ondat', 'presion_arterial_sistolica',
       'presion_arterial_diastolica', 'indice_mkillip', 'indice_killip',
       'frecuencia_cardiaca', 'diabetes_mellitus',
       'insuficiencia_cardiaca_congestiva', 'hipertension_arterial',
       'hiperlipoproteinemia', 'enfermedad_arterias_coronarias',
       'infarto_miocardio_agudo', 'fibrilacion_auricular',
       'intervencion_coronaria_percutanea', 'cabg', 'tabaquismo',
       'enfermedad_venosa_periferica', 'insuficiencia_renal_cronica',
       'dialisis', 'enfermedad_cerebro_vascu

In [56]:
new_table = original_table[columns].dropna(axis=0).reset_index(drop=True)
new_table.describe()

,edad,peso,talla,imc,sexo,color_piel,asa,clopidogrel,heparina,estatinas,...,TV/FV/PCR,Isquemia,ICC,Arritmia,creatinina,filtrado_glomerular,glicemia,ada,primera_asistencia_medica,estado_vital
count,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,...,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000,1256.000000
mean,66.025478,73.157643,168.117038,25.825707,0.685510,0.906051,5.010350,0.960191,0.993631,0.980096,...,0.087580,0.209395,0.282643,0.129777,102.531051,65.876380,7.583599,0.484076,199.944268,0.092357
std,12.612306,11.709661,5.777949,3.635491,0.464498,0.291874,4.844092,0.195588,0.079586,0.139728,...,0.282795,0.407039,0.450464,0.336192,59.334307,24.787143,3.513755,0.499945,219.356727,0.289644
min,13.000000,40.000000,120.000000,16.135000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,38.000000,3.115000,3.300000,0.000000,4.000000,0.000000
25%,57.000000,65.000000,165.000000,23.148000,0.000000,1.000000,3.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,78.000000,47.251000,5.500000,0.000000,60.000000,0.000000
50%,66.000000,72.000000,168.000000,25.649000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,89.000000,64.949000,6.400000,0.000000,120.000000,0.000000
75%,75.000000,81.000000,172.000000,28.092750,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,113.000000,82.555750,8.400000,1.000000,180.000000,0.000000
max,96.000000,140.000000,184.000000,58.273000,1.000000,1.000000,48.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1036.000000,179.424000,43.000000,1.000000,1080.000000,1.000000


In [58]:
# Guardando datos rellenados 1
'''
Datos extraidos de las 62 caracter'isticas con 1256 registros v'alidos
'''
new_table.to_csv(f'{PATH_SAVE}completed_data__C02__01.csv')